# Introduction

Notebook to analyse the PyBossa taskruns from the Expert App Version 2.

# Load Libraries and Data

In [21]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

In [22]:
import pandas as pd
import seaborn as sns
import numpy as np

import modules.utils.firefox_dataset_p2 as fd

from sklearn.metrics import cohen_kappa_score

taskruns = fd.TaskRuns.read_expert_taskruns_df()

TaskRuns shape: (93, 11)


# Grouping Contributions By Time

In [23]:
taskruns['created'] = pd.to_datetime(taskruns['created'], yearfirst=True)
taskruns['created_month'] = taskruns.apply(lambda row: row['created'].month, axis=1)
taskruns['created_day'] = taskruns.apply(lambda row: row['created'].day, axis=1)
taskruns['created_hour'] = taskruns.apply(lambda row: row['created'].hour, axis=1)

grouped_trs = taskruns.groupby(by=['created_month','created_day','created_hour']).count()
grouped_trs

bug_id  user_id  task_id  created  \
created_month created_day created_hour                                      
2             18          20                 8        8        8        8   
                          21                37       37       37       37   
              19          1                 13       13       13       13   
                          14                17       17       17       17   
                          15                 6        6        6        6   
                          17                10       10       10       10   
                          18                 2        2        2        2   

                                        finish_time  user_ip  link  timeout  \
created_month created_day created_hour                                        
2             18          20                      8        8     8        8   
                          21                     37       37    37       37   
              19          1                      13       13    13       13   
                          14                     17       17    17       17   
                          15                      6        6     6        6   
                          17                     10       10    10       10   
                          18                      2        2     2        2   

                                        project_id  id  answers  
created_month created_day created_hour                           
2             18          20                     8   8        8  
                          21                    37  37       37  
              19          1                     13  13       13  
                          14                    17  17       17  
                          15                     6   6        6  
                          17                    10  10       10  
                          18                     2   2        2

# Count TaskRuns by Task

In [24]:
cols = ['task_id','id']

df = taskruns[cols].groupby(by='task_id').count()
df.rename(columns={"id" :'count_trs'}, inplace=True)
display(df.tail(10))
print(df.shape)

,count_trs
task_id,
1804,1
1805,1
1806,1
1807,1
1808,1
1809,1
1810,1
1811,1
1812,1


(93, 1)


# Check All Tasks Have At Least One Answer

In [25]:
print(set(df.index) - set(range(1928,2020+1,1)))

{1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813}


# Analysis of Taskruns Infos

In [26]:
taskruns[['bug_id','answers']].head(10)

,bug_id,answers
0,1181835,0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,1248267,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2,1248268,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,1257087,1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,1264988,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
5,1267480,0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
6,1267501,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
7,1269348,0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
8,1269485,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
9,1270274,0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [27]:
taskruns.dtypes

bug_id                    int64
user_id                   int64
task_id                   int64
created          datetime64[ns]
finish_time              object
user_ip                  object
link                     object
timeout                  object
project_id                int64
id                        int64
answers                  object
created_month             int64
created_day               int64
created_hour              int64
dtype: object

# Compare Order of Taskruns

The order of the taskruns must be the same, ordered by the finish time or by the bug_id fields.

In [28]:
taskruns_ordered_by_finish_time = taskruns.sort_values(by='finish_time')
taskruns_ordered_by_bug_id = taskruns.copy()  ## already ordered by bug_id

diffs = taskruns_ordered_by_finish_time.bug_id - taskruns_ordered_by_bug_id.bug_id
d = np.sum(diff)

assert d == 0